# Experiment Database Storage Script

Adds targeted data directory as into project sqlite database tables "raw_data" and "metadata".

Data is processed just enough to save the structure of the directory (e.g., file_id, experiment_id, run_id). Optionally, one can add an "experiment_alias" tag, unique to this script, to metadata of all files stored by this script. This can be overwritten at any time by re-running with the metadata overwrite_flag keyword below set to "yes" or "force".

For visualizing and processing data, check for a "process_experiment_in_database.ipynb" file, probably in the same directory as this script.

As a result of the minimal processing, there is probably no reason to re-run this script unless one wishes to change the experiment_alias or reflect new changes to the data directory. In this case, make sure to check the overwrite_flag keyword argument below to ensure the database is properly updated.

---
TABLE: raw_data

new columns (in addition to csv columns) added by this script:
- file_id (new 1st column)
- file_row (new 2nd column)
---
TABLE: metadata

table columns: file_id, key, value

new k:v pairs (in addition to those in csv header) added by this script:
- file_id:  id unique to this file
- experiment_id:  id unique to this directory
- experiment_alias:  optional metadata tag for all files added by this script
- run_id:         id assigned to each folder of csv's by this script
- index_2d:       optional file order within a run, taken from filename after "Ind_" if present
- filepath:       filepath (absolute)
- relative_filepath:  filepath (relative to this script's directory)
- last_modified:  csv file-last-modified timestamp
---
id value dependencies:
- file_id derived from start time and filename (NOT filepath)
- experiment_id derived from this script's directory
---
assumptions made by script:
- Data is from __June 2019 or later__ (needs timestamps, header height, labtime)

In [1]:
experiment_alias = "RepTestRSA_181102"

# Imports and global variables

In [2]:
# General imports
from IPython.display import display
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import sqlite3 as sql

import eda.analysis.dataframe_plotting as dfplot
import eda.analysis.dataframe_processing as dfproc
import eda.data_io.csv_to_dataframe as csv2df
import eda.data_io.sqlite_io as sqlio

# CSV Import

In [3]:
# REQUIRED: CSV data & sqlite database directories
parent_dir = ('..\\..\\used_data')
db_parent_dir =  '..\\..\\..\\..\\database\\'
db_filepath = db_parent_dir + 'dnpdb.sqlite'

# REQUIRED: source csv format information
delimiter = '\t'
trailing_delimiters = True  # rows end in delimiters
num_headerlines = 9
column_names_row = 0  # NOT counting skipped lines from num_headerlines
pandas_read_csv_kwargs = {
     'skiprows': num_headerlines,
     'header': column_names_row,
     'names': None,
     'delimiter': delimiter,
}
if trailing_delimiters:  # needed to avoid problems
    pandas_read_csv_kwargs['index_col'] = False

# REQUIRED: filename key for data files, will load only these
filename_key = '.dat'

# REQUIRED: criteria for grouping runs
run_criteria = 'directory'

# OPTIONAL: filepath parsing rules
# DEFAULT SEARCH TERMS AND SEARCH RULES:
# 1. If first string found, register second string as
#    tag containing third string/value
#        e.g. if keyword_list contains ("warmup", "Warmup?", "Yes"):
#             "...warmup..." -> {"Warmup?": "Yes"}
this_element_keyword_list = [
]
# 2. Grab next element(s) if this one CONTAINS first string,
#    tag next element(s) as second string(s)
#        e.g. "..._Ind_3_..." -> {"FastScanIndex": 3}
#        e.g. "..._2Dscan_MirrorY_MirrorZ_..."
#                 -> {"SecondScanType": "MirrorY",
#                     "FirstScanType": "MirrorZ"}
next_element_keyword_list = [
    (   "Ind", "SecondScanIndex"),
    ("2Dscan", ["SecondScanType", "FirstScanType"])
]

# 3. Grab this element if it CONTAINS first string,
#    tag remainder as second string
#        e.g. "..._30K_..." -> {"SetTemperature": 30}
in_this_element_keyword_list = [
    (    "mT", "Magnetic Field (mT)"),
    (     "T", "Magnetic Field (T)"),
    (     "K", "Set Temperature (K)"),
    (    "nm", "Wavelength (nm)"),
    (    "ps", "Delay Time (ps)"),
    (     "V", "Voltage (V)"),
    (   "rep", "Reps per Step (#)"),
    (    "uW", "Pump Power (uW)"),
    ("uWPump", "Pump Power (uW)"),
    (    "mW", "Pump Power (mW)"),
    ("mWPump", "Pump Power (mW)"),
    (     "x", "SecondScanCoord"),
]

def process_secondscancoord(metadata_dict):
    if {'SecondScanType', 'SecondScanCoord'}.issubset(metadata_dict.keys()):
        key = metadata_dict['SecondScanType']
        metadata_dict[key] = metadata_dict['SecondScanCoord']
def consolidate_aliases_for_b_external(metadata_dict):
    if {'SecondScanType', 'SecondScanCoord'}.issubset(metadata_dict.keys()):
        if (metadata_dict['SecondScanType'] == 'Magnetic Field (T)'
                or metadata_dict['SecondScanType'] == 'BExternal'):
            metadata_dict['SecondScanType'] = 'Magnetic Field (mT)'
            metadata_dict['SecondScanCoord'] *= 1000.0
    if 'BExternal' in metadata_dict.keys():
        metadata_dict['Magnetic Field (T)'] = metadata_dict['BExternal']
        del metadata_dict['BExternal']
    if 'Magnetic Field (T)' in metadata_dict.keys():
        metadata_dict['Magnetic Field (mT)'] = \
            1000.0 * metadata_dict['Magnetic Field (T)']
        del metadata_dict['Magnetic Field (T)']
def consolidate_aliases_for_pump_power(metadata_dict):
    if 'Pump Power (uW)' in metadata_dict.keys():
        metadata_dict['Pump Power (mW)'] = \
            1e-3 * metadata_dict['Pump Power (uW)']
        del metadata_dict['Pump Power (uW)']
def handle_standalone_1d_scans(metadata_dict):
    if 'SecondScanIndex' not in metadata_dict:
        try:  # use persistent attribute of this function object
            handle_standalone_1d_scans.counter += 1
        except AttributeError:
            handle_standalone_1d_scans.counter = 1
        custom_ssi = handle_standalone_1d_scans.counter - 1
        metadata_dict['Run ID'] = -1
        metadata_dict['SecondScanIndex'] = custom_ssi
        metadata_dict['SecondScanType'] = "None"
        metadata_dict['SecondScanCoord'] = 0
#         metadata_dict['Magnetic Field (mT)'] = 0  # fixes issue later, but why?
        # get scan type, assume form "[type]_x_to_y.dat
        scan_type = metadata_dict['Filepath'].split('_')[-4]
        metadata_dict['FirstScanType'] = scan_type
def ensure_integer_index_2d(metadata_dict):
    if 'SecondScanIndex' in metadata_dict:
        metadata_dict['SecondScanIndex'] = int(metadata_dict['SecondScanIndex'])

# full metadata processing:
parsing_keyword_lists = [this_element_keyword_list,
                         next_element_keyword_list,
                         in_this_element_keyword_list]
metadata_processing_fcns = [process_secondscancoord,
                            consolidate_aliases_for_b_external,
                            consolidate_aliases_for_pump_power,
                            handle_standalone_1d_scans,
                            ensure_integer_index_2d]

# # alternatively, minimal processing version:
# parsing_keyword_lists = [[],[("Ind","SecondScanIndex")],[]]
# metadata_processing_fcns = []

metadata_filter_fcns = []
metadata_tag_to_column_list = []
dataframe_processing_fcns = []

# def handle_standalone_1d_scans(metadata_dict):
#     if 'SecondScanIndex' not in metadata_dict:
#         try:  # use persistent attribute of this function object
#             handle_standalone_1d_scans.counter += 1
#         except AttributeError:
#             handle_standalone_1d_scans.counter = 1
#         custom_ssi = handle_standalone_1d_scans.counter - 1
#         metadata_dict['Run ID'] = -1
#         metadata_dict['SecondScanIndex'] = custom_ssi
# def ensure_integer_index_2d(metadata_dict):
#     if 'SecondScanIndex' in metadata_dict:
#         metadata_dict['SecondScanIndex'] = int(metadata_dict['SecondScanIndex'])
# metadata_processing_fcns = [handle_standalone_1d_scans,
#                             ensure_integer_index_2d]

file_df_list, file_metadata_list = \
    csv2df.process_directory_csvs_to_dataframes(
        parent_dir, filename_key, run_criteria,
        pandas_read_csv_kwargs,
        parsing_keyword_lists,
        metadata_processing_fcns,
        metadata_filter_fcns,
        metadata_tag_to_column_list,
        dataframe_processing_fcns)

In [4]:
# Full df first line for column reference
orig_colnames = list(file_df_list[0])
file_df_list[1].head(1)

,scancoord,lockin2x,lockin1x,lockin2r,lockin1r,laserpower,cwetalon,lockin3x,lockin3r,lockin4x,lockin4r,lockin5x,lockin5r,lasercomponent1,lasercomponent2,temperature,labtime,file_index
0,-0.3,-0.0179,0.000873,0.01845,0.000887,-0.134,0.769,0.000628,0.000286,-0.0081,0.0145,-0.26288,0.26318,0.003,0.001,10.021,1.578671,1


In [5]:
file_metadata_df_list = []
for file_df, file_metadata in zip(file_df_list,
                                  file_metadata_list):
    updated_metadata = sqlio.get_updated_metadata_dict(file_metadata,
                                                       experiment_alias)
    file_metadata_df = sqlio.dict_to_two_column_df(updated_metadata)
    file_df.reset_index(inplace=True)
    file_df.rename(index=str, columns={'index': 'file_row'}, inplace=True)
    file_df.insert(0, 'file_id', updated_metadata['file_id'])
    file_metadata_df.insert(0, 'file_id', updated_metadata['file_id'])
    file_metadata_df_list.append(file_metadata_df)
    del file_df['file_index']

In [6]:
file_df_list[0].head(2)

,file_id,file_row,scancoord,lockin2x,lockin1x,lockin2r,lockin1r,laserpower,cwetalon,lockin3x,lockin3r,lockin4x,lockin4r,lockin5x,lockin5r,lasercomponent1,lasercomponent2,temperature,labtime
0,b'tzdTgY4JcTf7MegMM1dmfw==',0,-0.3,0.0220,0.001026,0.02725,0.001008,-0.137,0.769,0.000712,0.000731,-0.00400,0.00535,-0.29582,0.29616,0.311,0.001,10.017,1.631405
1,b'tzdTgY4JcTf7MegMM1dmfw==',1,0.0,0.0411,0.000944,0.04240,0.000953,-0.140,0.776,-0.000026,0.000822,-0.00505,0.00500,-0.29696,0.29730,0.006,-0.330,10.015,2.846728


In [7]:
print(list(file_metadata_df_list[0].key))  # print() for vertical space

['file_id', 'experiment_id', 'experiment_alias', 'run_id', 'index_2d', 'filepath', 'relative_filepath', 'last_modified', 'Delay Time (ps)', 'Wavelength (nm)', 'Set Temperature (K)', 'SecondScanType', 'FirstScanType', 'SecondScanCoord', 'Acquisition Program', 'Lockin Sensitivities (V)', 'Lockin Time Constants (s)', 'Lockin Reference Frequencies (Hz)', 'Lockin Reference Phases (degrees)', 'Scan Start', 'Scan End', 'NumReps']


In [8]:
# DEBUG TOOL: change row, remove preprocessed metadata from third file metadata
file_metadata_df_list[2] = file_metadata_df_list[2][
    file_metadata_df.key.isin(['file_id', 'experiment_id', 'experiment_alias',
                               'run_id', 'index_2d', 'filepath',
                               'relative_filepath', 'last_modified',
                               'Acquisition Program',
                               'Lockin Sensitivities (V)',
                               'Lockin Time Constants (s)',
                               'Lockin Reference Frequencies (Hz)',
                               'Lockin Reference Phases (degrees)',
                               'Scan Start', 'Scan End'])
    ]
file_metadata_df_list[2].value.iloc[2]

# Connect to database and update tables 'raw_data', 'metadata_df'

In [20]:
conn = sql.connect(db_filepath)  # make sure to close()!
sqlio.create_missing_tables(conn)  # database needs tables raw_data, metadata

In [19]:
num_file_additions = 0
for file_df in file_df_list:
    if sqlio.add_file_df_to_raw_data_if_missing(file_df, conn,
                                                overwrite_flag='warn'):
        num_file_additions += 1
print("Added %d files to table 'raw_data'!" % num_file_additions)

num_metadata_additions = 0
for metadata_df in file_metadata_df_list:
    if sqlio.update_metadata_table_from_metadata_df(metadata_df, conn,
                                                    overwrite_flag='warn'):
        num_metadata_additions += 1
print("Updated/added %d files in table 'metadata'!" % num_metadata_additions)

ProgrammingError: Cannot operate on a closed database.

In [11]:
# # DEBUG TOOL: block running close command
# raise Exception("don't close yet!")

In [22]:
# Always run at end!
conn.close()

# Debug Tools

In [13]:
# # DEBUG TOOL: add rows to a metadata_df
# ind = 0
# file_metadata_df_list[ind] = \
#     file_metadata_df_list[ind].append(file_metadata_df_list[ind].iloc[-1])
# file_metadata_df_list[ind].iloc[-1].key = "what"
# file_metadata_df_list[ind].iloc[-1].value = "up"

In [14]:
# # DEBUG TOOL: check what exists in table raw_data
# file_id = file_metadata_df_list[0].file_id.iloc[0]
# query = """SELECT *
#            FROM raw_data
#            WHERE file_id = ?"""
# qdf = pd.read_sql_query(query, conn, params=(file_id,))
# qdf

In [15]:
# # DEBUG TOOL: check what exists in table metadata
# file_id = file_metadata_df_list[0].file_id.iloc[0]
# query = """SELECT *
#            FROM metadata
#            WHERE file_id = ?"""
# qdf = pd.read_sql_query(query, conn, params=(file_id,))
# qdf

In [16]:
# # DEBUG TOOL: DELETE ALL ROWS FROM raw_data
# query = """DELETE FROM raw_data;"""
# conn.execute(query)
# conn.commit()
# query = """VACUUM;"""
# conn.execute(query)
# conn.commit()

In [21]:
# # DEBUG TOOL: DELETE ALL ROWS FROM metadata
# query = """DELETE FROM metadata;"""
# conn.execute(query)
# conn.commit()
# query = """VACUUM;"""
# conn.execute(query)
# conn.commit()